In [1]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import pyogrio
import numpy as np
import plotly.io as pio

In [2]:
pio.templates.default = "plotly_dark"

### Settings

In [3]:
sampling_factor = 0.01

output_path = "../../output/zones" # ptasc-2"
output_prefix = "mun_1pct_"

data_path = "../../analysis/data"

### Zoning data

In [4]:
zones = [
    { "id": "munich", "label": "München", "path": ["munich_area.gpkg"] },
    { "id": "umland", "label": "Münchner Umland", "path": ["umland_area.gpkg"] },
    { "id": "mvv", "label": "MVV Gebiet", "path": ["mvv_area.gpkg"] },
    { "id": "mvg_influence", "label": "MVG Einfluss", "path": ["mvg_influence_area.gpkg"] },
    { "id": "mvg_planning", "label": "MVG Planung", "path": ["mvg_planning_area.gpkg"] },
    { "id": "munich+umland", "label": "München & Umland", "path": ["munich_area.gpkg", "umland_area.gpkg"] },
]

df_zones = []
for zone in zones:
    df_partial = pd.concat([
        gpd.read_file("{}/{}".format(data_path, path))[["geometry"]]
        for path in zone["path"]]).dissolve()

    df_partial["zone_id"] = zone["id"]
    df_partial["zone_label"] = zone["label"]
    df_zones.append(df_partial)

df_zones = pd.concat(df_zones)

### Data preparation

In [5]:
# Load home locations
df_homes = pyogrio.read_dataframe("{}/{}homes.gpkg".format(output_path, output_prefix))

# Create a data frame to attach zone tags 
df_home_zones = gpd.sjoin(df_homes, df_zones, predicate = "within")[["household_id", "zone_id"]]

def home_zones(df, zones):
    df_selection = df_home_zones[df_home_zones["zone_id"].isin(zones)]
    return pd.merge(df, df_selection, on = "household_id")

In [6]:
# Load person information
df_persons = pd.read_csv("{}/{}persons.csv".format(output_path, output_prefix), sep = ";")

In [7]:
# Merge household information
df_households = pd.read_csv("{}/{}households.csv".format(output_path, output_prefix), sep = ";")
df_persons = pd.merge(df_persons, df_households, on = "household_id")

In [8]:
import shapely.geometry as sgeo
df_trips = pd.read_csv("{}/eqasim_trips.csv".format(output_path), sep = ";")

origin = gpd.points_from_xy(df_trips["origin_x"], df_trips["origin_y"])
destination = gpd.points_from_xy(df_trips["destination_x"], df_trips["destination_y"])

df_trips["geometry"] = [sgeo.LineString(od) for od in zip(origin, destination)]
df_trips = gpd.GeoDataFrame(df_trips, crs = "EPSG:25832")

In [9]:
# Load trip information
#df_trips = pyogrio.read_dataframe("{}/{}trips.gpkg".format(output_path, output_prefix))
df_trips["trip_index"] = np.arange(len(df_trips))

# Merge in household id
df_trips = pd.merge(df_trips, df_persons[["person_id", "household_id"]])

# Create a data frame to attach zone tags 
# TODO: Should this be within or intersects? Another option is to use the home zone
#df_trip_zones = gpd.sjoin(df_trips[["trip_index", "geometry"]], df_zones, predicate = "within")[[
#    "trip_index", "zone_id"
#]]

df_trip_zones  = df_trips[["person_id", "trip_index"]].copy()
df_trip_zones = pd.merge(df_trip_zones, df_persons[["person_id", "household_id"]], on = "person_id")
df_trip_zones = pd.merge(df_trip_zones, df_home_zones, on = "household_id").drop(columns = ["person_id"])

def trip_zones(df, zones):
    df_selection = df_trip_zones[df_trip_zones["zone_id"].isin(zones)]
    return pd.merge(df, df_selection, on = "trip_index")

In [10]:
# Add tag if person is active
df_persons["is_active"] = df_persons["person_id"].isin(df_trips["person_id"])

In [11]:
# Enrich trips
df_trips["trips"] = 1.0
df_trips["distance_km"] = df_trips["routed_distance"] * 1e-3
df_trips["travel_time_min"] = df_trips["travel_time"] / 60
df_trips["euclidean_distance_km"] = df_trips["euclidean_distance"] * 1e-3

# Definition of purpose
df_trips["purpose"] = df_trips["following_purpose"]
f_return = df_trips["following_purpose"] == "home"
df_trips.loc[f_return, "purpose"] = df_trips.loc[f_return, "preceding_purpose"]

# Remove zero distance trips
df_trips = df_trips[df_trips["euclidean_distance"] > 0]

# MiD Comparison

In [12]:
df = trip_zones(df_trips[["trip_index", "mode", "distance_km", "travel_time_min"]], ["mvv", "munich", "umland"]).drop(columns = ["trip_index"]).groupby(
    ["zone_id", "mode"]).median(numeric_only = True).reset_index().assign(data = "synthetic")

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "mode": "walk", "distance_km": 1, "travel_time_min": 1, "data": "MiD" },
    { "zone_id": "mvv", "mode": "bike", "distance_km": 2, "travel_time_min": 2, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car_passenger", "distance_km": 6, "travel_time_min": 6, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car", "distance_km": 8, "travel_time_min": 8, "data": "MiD" },
    { "zone_id": "mvv", "mode": "pt", "distance_km": 8, "travel_time_min": 8, "data": "MiD" },
    { "zone_id": "munich", "mode": "walk", "distance_km": 1, "travel_time_min": 1, "data": "MiD" },
    { "zone_id": "munich", "mode": "bike", "distance_km": 2, "travel_time_min": 2, "data": "MiD" },
    { "zone_id": "munich", "mode": "car_passenger", "distance_km": 6, "travel_time_min": 6, "data": "MiD" },
    { "zone_id": "munich", "mode": "car", "distance_km": 7, "travel_time_min": 7, "data": "MiD" },
    { "zone_id": "munich", "mode": "pt", "distance_km": 6, "travel_time_min": 6, "data": "MiD" },
    { "zone_id": "umland", "mode": "walk", "distance_km": 1, "travel_time_min": 1, "data": "MiD" },
    { "zone_id": "umland", "mode": "bike", "distance_km": 2, "travel_time_min": 2, "data": "MiD" },
    { "zone_id": "umland", "mode": "car_passenger", "distance_km": 6, "travel_time_min": 6, "data": "MiD" },
    { "zone_id": "umland", "mode": "car", "distance_km": 8, "travel_time_min": 8, "data": "MiD" },
    { "zone_id": "umland", "mode": "pt", "distance_km": 14, "travel_time_min": 14, "data": "MiD" },
])])

In [13]:
px.bar(
    df, x = "zone_id", y = "distance_km", pattern_shape = "data", barmode = "group",
    title = "Median distance by mode", color = "mode"
)

In [14]:
px.bar(
    df, x = "zone_id", y = "travel_time_min", pattern_shape = "data", barmode = "group",
    title = "Median travel time by mode", color = "mode"
)

### Mode share by trips and distance (Figure 17)

In [15]:
df = trip_zones(df_trips[["trip_index", "mode"]], ["mvv", "munich", "umland"]).drop(columns = ["trip_index"]).groupby([
    "mode", "zone_id"
]).size().reset_index(name = "count").assign(data = "synthetic")

df_total = df.groupby("zone_id")["count"].sum().reset_index(name = "total")
df = pd.merge(df, df_total, on = "zone_id")
df["share"] = df["count"] / df["total"]

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "mode": "walk", "share": 0.21, "data": "MiD" },
    { "zone_id": "mvv", "mode": "bike", "share": 0.15, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car_passenger", "share": 0.12, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car", "share": 0.34, "data": "MiD" },
    { "zone_id": "mvv", "mode": "pt", "share": 0.18, "data": "MiD" },
    { "zone_id": "munich", "mode": "walk", "share": 0.24, "data": "MiD" },
    { "zone_id": "munich", "mode": "bike", "share": 0.18, "data": "MiD" },
    { "zone_id": "munich", "mode": "car_passenger", "share": 0.10, "data": "MiD" },
    { "zone_id": "munich", "mode": "car", "share": 0.24, "data": "MiD" },
    { "zone_id": "munich", "mode": "pt", "share": 0.24, "data": "MiD" },
    { "zone_id": "umland", "mode": "walk", "share": 0.18, "data": "MiD" },
    { "zone_id": "umland", "mode": "bike", "share": 0.13, "data": "MiD" },
    { "zone_id": "umland", "mode": "car_passenger", "share": 0.14, "data": "MiD" },
    { "zone_id": "umland", "mode": "car", "share": 0.44, "data": "MiD" },
    { "zone_id": "umland", "mode": "pt", "share": 0.11, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "share", pattern_shape = "data", barmode = "group",
    title = "Mode share by trips", color = "mode"
)